In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [2]:
# Montando o Google Drive no ambiente do Google Colab
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
# Definindo os caminhos dos arquivos CSV de exportação e importação
file_path = '/content/drive/MyDrive/projeto_dnc/df_exp_tratado.xlsx'

In [4]:
df = pd.read_excel(file_path)

In [5]:
# Removendo valores ausentes e duplicados
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)

In [6]:
df.head()

,Unnamed: 0,CO_ANO,CO_MES,CO_NCM,CO_PAIS,SG_UF_NCM,KG_LIQUIDO,VL_FOB,Categoria,Produto,Tipo,Origem,Setor
0,0,2024,8,64039990,301,CE,989,25361,Calçados e artigos de calçados,Partes de Calçados,Calçados,Vegetal,Agricultura
1,1,2024,8,24012030,87,SC,462000,4236540,Tabaco e seus substitutos manufaturados,Tabaco não manufaturado; restos de tabaco,Tabaco,Tabaco,Agricultura
2,2,2024,10,33011290,249,SP,2067847,29292037,"Óleos Essenciais, perfumaria, cosmética, higie...","Óleos; essenciais (concretos, absolutos); seus...",Óleos Essenciais,Vegetal,Agricultura
3,3,2024,1,41044130,858,GO,27469,320819,Couros e peles,Couros e peles curtidos ou em crosta de bovino...,Couro,Bovinos,Pecuária
4,4,2024,10,62171000,580,SP,20,75,Vestuário e Acessórios de Vestuário,Roupas,Roupas,Vegetal,Agricultura


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 423632 entries, 0 to 423631
Data columns (total 13 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  423632 non-null  int64 
 1   CO_ANO      423632 non-null  int64 
 2   CO_MES      423632 non-null  int64 
 3   CO_NCM      423632 non-null  int64 
 4   CO_PAIS     423632 non-null  int64 
 5   SG_UF_NCM   423632 non-null  object
 6   KG_LIQUIDO  423632 non-null  int64 
 7   VL_FOB      423632 non-null  int64 
 8   Categoria   423632 non-null  object
 9   Produto     423632 non-null  object
 10  Tipo        423632 non-null  object
 11  Origem      423632 non-null  object
 12  Setor       423632 non-null  object
dtypes: int64(7), object(6)
memory usage: 42.0+ MB


In [8]:
# Selecionando apenas as colunas relevantes para análise de exportação e importação
df = df[['CO_NCM', 'Origem', 'CO_ANO', 'VL_FOB']]

In [9]:
# Agrupando os dados de exportação por NCM somando os valores FOB
df_exp = df.groupby(['CO_NCM', 'Origem', 'CO_ANO'], as_index=False)['VL_FOB'].sum()

In [10]:
df_exp.head()

,CO_NCM,Origem,CO_ANO,VL_FOB
0,10011100,Trigo,2024,26417
1,10011900,Trigo,2024,31
2,10019100,Trigo,2024,26136
3,10019900,Trigo,2024,608108402
4,10029000,Centeio,2024,13433


In [11]:
df_exp_2023 = pd.read_csv('/content/drive/MyDrive/projeto_dnc/EXP_2023.csv', sep=';')

In [12]:
df_exp_2023.head()

,CO_ANO,CO_MES,CO_NCM,CO_UNID,CO_PAIS,SG_UF_NCM,CO_VIA,CO_URF,QT_ESTAT,KG_LIQUIDO,VL_FOB
0,2023,3,84139190,10,160,RJ,4,817700,107365,5,20
1,2023,12,73182100,10,764,SP,1,817800,0,0,84
2,2023,4,84219999,10,40,RS,4,817600,2,2,758
3,2023,5,85291090,10,63,RJ,7,1017503,1,1,275
4,2023,10,84825010,11,63,MG,7,1017503,2,0,9


In [13]:
df_exp_2024 = df_exp.copy()

In [14]:
df_exp_2024.head()

,CO_NCM,Origem,CO_ANO,VL_FOB
0,10011100,Trigo,2024,26417
1,10011900,Trigo,2024,31
2,10019100,Trigo,2024,26136
3,10019900,Trigo,2024,608108402
4,10029000,Centeio,2024,13433


In [15]:
# Selecionando apenas as colunas relevantes para análise de exportação e importação
df_exp_2023 = df_exp_2023[['CO_NCM', 'CO_ANO', 'VL_FOB']].reset_index(drop=True)

In [16]:
df_exp_2023.head()

,CO_NCM,CO_ANO,VL_FOB
0,84139190,2023,20
1,73182100,2023,84
2,84219999,2023,758
3,85291090,2023,275
4,84825010,2023,9


In [17]:
# Fazendo merge com base na coluna CO_NCM
df_exp_2023 = df_exp_2023.merge(
    df_exp[['CO_NCM', 'Origem']],
    on='CO_NCM',
    how='left'
)

In [18]:
df_exp_2023.head()

,CO_NCM,CO_ANO,VL_FOB,Origem
0,84139190,2023,20,NaN
1,73182100,2023,84,NaN
2,84219999,2023,758,NaN
3,85291090,2023,275,NaN
4,84825010,2023,9,NaN


In [19]:
df_exp_2023 = df_exp_2023.dropna().reset_index(drop=True)

In [20]:
df_exp_2023.rename(columns={'VL_FOB': 'VL_FOB_2023'}, inplace=True)
df_exp_2024.rename(columns={'VL_FOB': 'VL_FOB_2024'}, inplace=True)

In [21]:
# Agrupando os dados de exportação por NCM somando os valores FOB
df_exp_2024 = df_exp_2024.groupby(['Origem'], as_index=False)['VL_FOB_2024'].sum()
df_exp_2023 = df_exp_2023.groupby(['Origem'], as_index=False)['VL_FOB_2023'].sum()

In [22]:
label_dict = {
    'Soja': 1,
    'Cana de Açúcar': 2,
    'Madeira': 3,
    'Milho': 4,
    'Algodão': 5,
    'Tabaco': 6,
    'Vegetal': 7,
    'Animal': 8,
    'Trigo': 9,
    'Bovinos': 10,
    'Amendoim': 11,
    'Oleaginosas': 12,
    'Vegetal e Animal': 13,
    'Cacau': 14,
    'Arroz': 15,
    'Malte': 16,
    'Leveduras': 17,
    'Plantas ': 18,
    'Sorgo': 19,
    'Amido': 20,
    'Colza': 21,
    'Peixe': 22,
    'Cereais': 23,
    'Mandioca': 24,
    'Animais': 25,
    'Palma': 26,
    'Uva': 27,
    'Aveia': 28,
    'Aves': 29,
    'Centeio': 30,
    'Azeitona': 31,
    'Lã': 32,
    'Tomate': 33,
    'Girassol': 34,
    'Cevada': 35,
    'Madeira': 36
}

In [23]:
# Aplica a codificação
df_exp_2023['Origem_codificada'] = df_exp_2023['Origem'].map(label_dict)
df_exp_2024['Origem_codificada'] = df_exp_2024['Origem'].map(label_dict)

In [24]:
df_exp_2023.head()

,Origem,VL_FOB_2023,Origem_codificada
0,Algodão,3205148214,5
1,Amendoim,625359219,11
2,Amido,64605374,20
3,Animais,22472696,25
4,Animal,2349734035,8


In [25]:
df_exp_2024.head()

,Origem,VL_FOB_2024,Origem_codificada
0,Algodão,5289442446,5
1,Amendoim,467769469,11
2,Amido,62198479,20
3,Animais,18510994,25
4,Animal,2249960225,8


In [26]:
# Converter a coluna para int (truncando)
df_exp_2024['Origem_codificada'] = df_exp_2024['Origem_codificada'].astype(int)
df_exp_2023['Origem_codificada'] = df_exp_2023['Origem_codificada'].astype(int)

In [27]:
# Calculando o valor total geral
total_exp_2024 = df_exp_2024['VL_FOB_2024'].sum()

# Criando uma nova coluna com a participação percentual de cada produto
df_exp_2024['PERC_PARTICIPACAO_EXP'] = ((df_exp_2024['VL_FOB_2024'] / total_exp_2024) * 100).round(2)

# Ordenando do maior para o menor
df_exp_2024 = df_exp_2024.sort_values(by='PERC_PARTICIPACAO_EXP', ascending=False)

In [28]:
# Calculando o valor total geral
total_exp_2023 = df_exp_2023['VL_FOB_2023'].sum()

# Criando uma nova coluna com a participação percentual de cada produto
df_exp_2023['PERC_PARTICIPACAO_EXP'] = ((df_exp_2023['VL_FOB_2023'] / total_exp_2023) * 100).round(2)

# Ordenando do maior para o menor
df_exp_2023 = df_exp_2023.sort_values(by='PERC_PARTICIPACAO_EXP', ascending=False)

In [29]:
df_exp_2024

,Origem,VL_FOB_2024,Origem_codificada,PERC_PARTICIPACAO_EXP
27,Soja,54140037484,1,42.76
11,Cana de Açúcar,20222626420,2,15.97
19,Madeira,16803729821,36,13.27
33,Vegetal,9691882998,7,7.65
22,Milho,8177075971,4,6.46
0,Algodão,5289442446,5,4.18
29,Tabaco,2975322951,6,2.35
4,Animal,2249960225,8,1.78
34,Vegetal e Animal,1773548798,13,1.40
9,Bovinos,1236311850,10,0.98


In [30]:
df_exp_2024 = df_exp_2024.merge(
    df_exp_2023[['Origem_codificada', 'VL_FOB_2023']],
    on='Origem_codificada',
    how='left'
)

In [31]:
df_exp_2024

,Origem,VL_FOB_2024,Origem_codificada,PERC_PARTICIPACAO_EXP,VL_FOB_2023
0,Soja,54140037484,1,42.76,67509187644
1,Cana de Açúcar,20222626420,2,15.97,17909021749
2,Madeira,16803729821,36,13.27,13819639007
3,Vegetal,9691882998,7,7.65,8752182375
4,Milho,8177075971,4,6.46,13613016930
5,Algodão,5289442446,5,4.18,3205148214
6,Tabaco,2975322951,6,2.35,2729473808
7,Animal,2249960225,8,1.78,2349734035
8,Vegetal e Animal,1773548798,13,1.40,1612794358
9,Bovinos,1236311850,10,0.98,1095717218


In [32]:
# 3. Calcular a diferença
df_exp_2024['DIF_VL_FOB'] = df_exp_2024['VL_FOB_2024'] - df_exp_2024['VL_FOB_2023']

In [33]:
df_exp_2024['PERC_DIF_VL_FOB'] = ((df_exp_2024['DIF_VL_FOB'] / df_exp_2024['VL_FOB_2023']) * 100).round(2)

In [34]:
df_exp_2024

,Origem,VL_FOB_2024,Origem_codificada,PERC_PARTICIPACAO_EXP,VL_FOB_2023,DIF_VL_FOB,PERC_DIF_VL_FOB
0,Soja,54140037484,1,42.76,67509187644,-13369150160,-19.80
1,Cana de Açúcar,20222626420,2,15.97,17909021749,2313604671,12.92
2,Madeira,16803729821,36,13.27,13819639007,2984090814,21.59
3,Vegetal,9691882998,7,7.65,8752182375,939700623,10.74
4,Milho,8177075971,4,6.46,13613016930,-5435940959,-39.93
5,Algodão,5289442446,5,4.18,3205148214,2084294232,65.03
6,Tabaco,2975322951,6,2.35,2729473808,245849143,9.01
7,Animal,2249960225,8,1.78,2349734035,-99773810,-4.25
8,Vegetal e Animal,1773548798,13,1.40,1612794358,160754440,9.97
9,Bovinos,1236311850,10,0.98,1095717218,140594632,12.83
